# Generating embeddings for SIB SPARQL queries

Using `fastembed` library, FlagEmbedding large 1.5 embedding model, and Qdrant vectorstore

Example code for loading the large Babel concepts synonyms dataset to Qdrant: https://github.com/vemonet/concept-resolver/blob/main/src/babel_load.py

Questions:
* Which info can I get from the UniProt endpoint?
* Give me an example to access cross references from the UniProt SPARQL endpoint to all the databases available in the endpoint

In [ ]:
from sparql_llm.index import init_vectordb

init_vectordb("localhost")


# get_schemaorg_description({
#     "label": "Bgee",
#     "endpoint": "https://www.bgee.org/sparql/",
#     "homepage": "https://www.bgee.org/",
# })